# 🏗️ HVAC AI Platform - One-Click Cloud Launcher

**Repository:** `https://github.com/elliotttmiller/hvac.git`

This notebook provides instant cloud deployment of the HVAC AI Platform on Google Colab.

---

## 📋 Features
- ✅ Automated environment setup (Node.js 20+ LTS, Python 3.11+)
- ✅ Repository cloning and dependency installation
- ✅ Secure API key configuration
- ✅ Public URL tunneling (localtunnel) for frontend & backend
- ✅ Full platform launch via `start.py`

---

## 🔐 Required Secrets
Before running, you'll need:
- `VITE_AI_API_KEY` or `VITE_GEMINI_API_KEY` - Your AI API key

---

## 🚀 Quick Start
1. **Run all cells sequentially** (Runtime → Run all)
2. **Enter your API keys** when prompted
3. **Access your app** via the printed tunnel URLs

---

## 🔧 Step 1: Mount Google Drive (Optional)

Mount your Google Drive to persist data and configurations.

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive', force_remount=False)

print("✅ Google Drive mounted successfully")
print(f"📂 Current directory: {os.getcwd()}")

## 🐍 Step 2: Verify Python Environment

Check Python version (should be 3.11+)

In [ ]:
import sys
import platform

print(f"🐍 Python Version: {sys.version}")
print(f"🖥️  Platform: {platform.system()} {platform.release()}")
print(f"📍 Python Executable: {sys.executable}")

# Verify Python 3.8+ (Colab typically has 3.10+)
version_info = sys.version_info
if version_info.major >= 3 and version_info.minor >= 8:
    print("✅ Python version is compatible")
else:
    print("❌ Python version must be 3.8 or higher")

## 📦 Step 3: Install Node.js 20+ LTS

Install Node.js via NodeSource repository for latest LTS version.

In [ ]:
%%bash

# Install Node.js 20.x LTS from NodeSource
echo "📦 Installing Node.js 20.x LTS..."

# Check if Node.js is already installed
if command -v node &> /dev/null; then
    CURRENT_VERSION=$(node --version)
    echo "ℹ️  Node.js already installed: $CURRENT_VERSION"
    
    # Check if version is 20+
    MAJOR_VERSION=$(echo $CURRENT_VERSION | cut -d'v' -f2 | cut -d'.' -f1)
    if [ "$MAJOR_VERSION" -ge 20 ]; then
        echo "✅ Node.js version is sufficient (v20+)"
        exit 0
    else
        echo "⚠️  Node.js version is too old, upgrading..."
    fi
fi

# Install Node.js 20.x
curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
sudo apt-get install -y nodejs

# Verify installation
echo "✅ Node.js installed:"
node --version
npm --version

## 📥 Step 4: Clone HVAC Repository

Clone the repository from GitHub.

In [ ]:
import os
import shutil

# Define repo details
REPO_URL = "https://github.com/elliotttmiller/hvac.git"
REPO_DIR = "/content/hvac"

# Remove existing directory if present
if os.path.exists(REPO_DIR):
    print(f"⚠️  Removing existing directory: {REPO_DIR}")
    shutil.rmtree(REPO_DIR)

# Clone repository
print(f"📥 Cloning repository from {REPO_URL}...")
!git clone {REPO_URL} {REPO_DIR}

# Change to repo directory
os.chdir(REPO_DIR)

print(f"✅ Repository cloned successfully")
print(f"📂 Current directory: {os.getcwd()}")

# Show directory contents
print("\n📁 Repository contents:")
!ls -la

## 🔐 Step 5: Configure API Keys

**CRITICAL:** Enter your API keys when prompted. These will be injected into `.env` file.

You can get API keys from:
- **Gemini:** https://makersuite.google.com/app/apikey
- **OpenAI:** https://platform.openai.com/api-keys

In [ ]:
from getpass import getpass
import os

# Prompt for API keys (hidden input)
print("🔐 API Key Configuration")
print("=" * 50)

# Check if user wants to use Gemini or other provider
ai_provider = input("\nSelect AI Provider (gemini/openai/anthropic) [gemini]: ").strip().lower() or "gemini"

# Get API key
if ai_provider == "gemini":
    api_key = getpass("Enter your GEMINI API Key: ")
    gemini_key = api_key
elif ai_provider == "openai":
    api_key = getpass("Enter your OPENAI API Key: ")
    gemini_key = ""
else:
    api_key = getpass("Enter your API Key: ")
    gemini_key = ""

# Optional: Gemini-specific key
if not gemini_key:
    gemini_prompt = input("\nAlso provide Gemini key? (y/n) [n]: ").strip().lower()
    if gemini_prompt == 'y':
        gemini_key = getpass("Enter your GEMINI API Key: ")

# Generate .env file
env_content = f"""# ============================================================================
# AI Provider Configuration (Auto-generated by Colab Launcher)
# ============================================================================

# AI Provider Selection
VITE_AI_PROVIDER={ai_provider}

# AI API Key
VITE_AI_API_KEY={api_key}

# Provider-specific keys
{f"VITE_GEMINI_API_KEY={gemini_key}" if gemini_key else "# VITE_GEMINI_API_KEY=your_gemini_key_here"}

# Model Selection
VITE_AI_MODEL={"gemini-2.5-flash" if ai_provider == "gemini" else "gpt-4o" if ai_provider == "openai" else "claude-3-5-sonnet-20241022"}

# AI Generation Parameters
VITE_AI_TEMPERATURE=0.2
VITE_AI_MAX_TOKENS=4096

# ============================================================================
# Feature Flags
# ============================================================================

VITE_FEATURE_CACHE=true
VITE_FEATURE_FILE_PROCESSING=true

# ============================================================================
# Rate Limiting
# ============================================================================

VITE_RATE_LIMIT_MAX_RETRIES=3
VITE_RATE_LIMIT_DELAY_MS=1000
VITE_RATE_LIMIT_EXPONENTIAL_BACKOFF=true

# ============================================================================
# File Processing
# ============================================================================

VITE_FILE_MAX_SIZE=10485760
VITE_FILE_SUPPORTED_FORMATS=pdf,png,jpg,jpeg,dwg
VITE_FILE_PDF_DPI=300
"""

# Write .env file
env_path = "/content/hvac/.env"
with open(env_path, "w") as f:
    f.write(env_content)

print("\n✅ .env file created successfully")
print(f"📄 Location: {env_path}")
print("\n⚠️  API keys are sensitive - never commit .env to version control!")

## 📦 Step 6: Install Dependencies

Install npm dependencies for the project.

In [ ]:
%%bash

cd /content/hvac

echo "📦 Installing npm dependencies..."
echo "⏱️  This may take 2-3 minutes..."

npm install

echo "✅ Dependencies installed successfully"

## 🌐 Step 7: Setup Port Tunneling

Install and configure `localtunnel` to expose ports 3000 (Frontend) and 4000 (Backend) to the internet.

In [ ]:
%%bash

# Install localtunnel globally
echo "🌐 Installing localtunnel..."
npm install -g localtunnel

echo "✅ Localtunnel installed successfully"

## 🚀 Step 8: Launch Application

**IMPORTANT:** This cell will start the application servers and create public URLs.

The cell will:
1. Run `start.py` to validate environment and start dev servers
2. Create public tunnels for frontend (port 3000) and backend (port 4000)
3. Print public URLs that you can access from any browser

**Note:** The servers will keep running. To stop them, use Runtime → Interrupt execution.

In [ ]:
import subprocess
import threading
import time
import os

os.chdir('/content/hvac')

print("🚀 Starting HVAC AI Platform...")
print("=" * 70)

# Start tunnels in background threads
tunnel_processes = []
tunnel_urls = {}

def start_tunnel(port, name):
    """Start a localtunnel for the given port"""
    try:
        proc = subprocess.Popen(
            ['lt', '--port', str(port)],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        tunnel_processes.append(proc)
        
        # Wait for URL to be printed
        for line in proc.stdout:
            if 'your url is:' in line.lower():
                url = line.split('your url is:')[-1].strip()
                tunnel_urls[name] = url
                print(f"\n🌐 {name} URL: {url}")
                break
    except Exception as e:
        print(f"❌ Failed to start tunnel for {name}: {e}")

# Start frontend tunnel
print("\n🌐 Creating public tunnel for Frontend (port 3000)...")
frontend_thread = threading.Thread(target=start_tunnel, args=(3000, 'Frontend'))
frontend_thread.daemon = True
frontend_thread.start()

# Wait a bit before starting backend tunnel
time.sleep(3)

# Start backend tunnel
print("🌐 Creating public tunnel for Backend (port 4000)...")
backend_thread = threading.Thread(target=start_tunnel, args=(4000, 'Backend'))
backend_thread.daemon = True
backend_thread.start()

# Wait for tunnels to establish
time.sleep(5)

print("\n" + "=" * 70)
print("🎉 TUNNELS READY!")
print("=" * 70)

# Now start the application
print("\n🚀 Starting application servers...")
print("📝 Running: python start.py --no-dev\n")

# Run start.py with --no-dev to do diagnostics without blocking
result = subprocess.run(
    ['python3', 'start.py', '--no-dev'],
    capture_output=False,
    text=True
)

if result.returncode == 0:
    print("\n✅ Diagnostics passed! Now starting dev servers...")
    
    # Start dev servers in background
    print("\n🚀 Starting Frontend & Backend servers...")
    
    # Start frontend
    frontend_proc = subprocess.Popen(
        ['npm', 'run', 'dev'],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )
    
    # Start backend (if exists)
    backend_proc = None
    if os.path.exists('server/index.cjs'):
        backend_proc = subprocess.Popen(
            ['npm', 'run', 'dev:api'],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )
    
    # Wait for servers to start
    print("⏱️  Waiting for servers to initialize (30s)...")
    time.sleep(30)
    
    print("\n" + "=" * 70)
    print("🎉 APPLICATION READY!")
    print("=" * 70)
    print("\n📍 Access your application:")
    print(f"   Frontend: {tunnel_urls.get('Frontend', 'Waiting...')}")
    print(f"   Backend:  {tunnel_urls.get('Backend', 'Waiting...')}")
    print("\n⚠️  Keep this cell running to maintain the tunnels and servers.")
    print("   To stop: Runtime → Interrupt execution")
    print("=" * 70)
    
    # Stream logs from frontend
    try:
        for line in frontend_proc.stdout:
            print(f"[Frontend] {line.strip()}")
    except KeyboardInterrupt:
        print("\n⚠️  Shutting down servers...")
        frontend_proc.terminate()
        if backend_proc:
            backend_proc.terminate()
        for proc in tunnel_processes:
            proc.terminate()
else:
    print("\n❌ Diagnostics failed! Check errors above.")

---

## 📝 Troubleshooting

### Application won't start
- Verify API keys are correct in Step 5
- Check that Node.js 20+ is installed (Step 3)
- Ensure npm dependencies installed successfully (Step 6)

### Tunnels not working
- Localtunnel can be rate-limited, try restarting Step 8
- Alternative: Use `ngrok` instead (requires account)

### Out of memory
- Google Colab free tier has limited RAM
- Try restarting runtime: Runtime → Restart runtime
- Consider upgrading to Colab Pro for more resources

---

## 📚 Additional Resources
- Repository: https://github.com/elliotttmiller/hvac
- Documentation: See `docs/` folder in repository
- Issues: https://github.com/elliotttmiller/hvac/issues

---

**Made with ❤️ for Cloud Portability**